In [1]:
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import yfinance as yf
from time import sleep

In [2]:
wiki = 'https://en.wikipedia.org/wiki/'

In [3]:
cac40_tickers = pd.read_html(wiki + 'CAC_40', flavor='html5lib')[4]['Ticker'].to_list()

In [4]:
def get_stock_price_history(tickers):
    data = []
    for ticker in tickers:
        data.append(yf.download(tickers=ticker,
                                period="5d",
                                interval="1m").reset_index())
    return data

In [29]:
cac40 = get_stock_price_history(cac40_tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [30]:
def new_engine(name):
    engine = create_engine(f'postgresql://pavelkurach@localhost:5432/{name}')
    if not database_exists(engine.url):
        create_database(engine.url)
    print(database_exists(engine.url))
    return engine

In [31]:
cac40_engine = new_engine('CAC40')

True


In [32]:
def to_sql(frames, tickers, engine):
    for frame, ticker in zip(frames, tickers):
        frame.to_sql(ticker, engine, if_exists='replace', index=False)
        with cac40_engine.connect() as con:
            con.execute(f'ALTER TABLE "{ticker}" ADD PRIMARY KEY ("{frame.columns[0]}");')

    print("Successfully created a database")

In [33]:
to_sql(cac40, cac40_tickers, cac40_engine)

Successfully created a database


In [34]:
Base = automap_base()

Base.prepare(autoload_with=cac40_engine)

In [35]:
cac40_classes = {ticker: cls for ticker, cls in zip(cac40_tickers, Base.classes)}

In [36]:
session = Session(cac40_engine)

In [37]:
query = session.query(cac40_classes['AI.PA'])

In [38]:
df = pd.read_sql(query.statement, query.session.bind, index_col='Datetime')
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-19 09:00:00,133.520004,133.600006,133.399994,133.399994,133.399994,0
2022-12-19 09:01:00,133.500000,133.500000,133.399994,133.479996,133.479996,964
2022-12-19 09:02:00,133.479996,133.580002,133.479996,133.559998,133.559998,527
2022-12-19 09:03:00,133.520004,133.559998,133.479996,133.520004,133.520004,526
2022-12-19 09:04:00,133.479996,133.600006,133.479996,133.580002,133.580002,646
...,...,...,...,...,...,...
2022-12-23 13:25:00,133.460007,133.460007,133.460007,133.460007,133.460007,2
2022-12-23 13:26:00,133.440002,133.440002,133.440002,133.440002,133.440002,133
2022-12-23 13:27:00,133.419998,133.419998,133.399994,133.399994,133.399994,227


In [39]:
import matplotlib

df.Close.pct_change()

Datetime
2022-12-19 09:00:00         NaN
2022-12-19 09:01:00    0.000600
2022-12-19 09:02:00    0.000599
2022-12-19 09:03:00   -0.000299
2022-12-19 09:04:00    0.000449
                         ...   
2022-12-23 13:25:00    0.000000
2022-12-23 13:26:00   -0.000150
2022-12-23 13:27:00   -0.000300
2022-12-23 13:28:00    0.000150
2022-12-23 13:29:00    0.000300
Name: Close, Length: 2260, dtype: float64

In [40]:
import numpy as np

highs = df['High'] / df['Close'].shift() - 1
print(highs)

Datetime
2022-12-19 09:00:00         NaN
2022-12-19 09:01:00    0.000750
2022-12-19 09:02:00    0.000749
2022-12-19 09:03:00    0.000000
2022-12-19 09:04:00    0.000599
                         ...   
2022-12-23 13:25:00    0.000000
2022-12-23 13:26:00   -0.000150
2022-12-23 13:27:00   -0.000150
2022-12-23 13:28:00    0.000150
2022-12-23 13:29:00    0.000300
Length: 2260, dtype: float64


In [41]:
lows = df['Low'] / df['Close'].shift() - 1
np.sum(lows < -3 * lows.std())

35

In [42]:
import datetime as dt

print(dt.datetime.now())
print(df.index[0] < dt.datetime.now())
start = df.index[-1]
start

2022-12-23 13:45:19.375647
True


Timestamp('2022-12-23 13:29:00')

In [43]:
yf.download(tickers='AI.PA',
            period="1d",
            interval="1m").reset_index()

[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-12-23 09:00:00,133.619995,133.839996,133.619995,133.740005,133.740005,0
1,2022-12-23 09:01:00,133.779999,133.919998,133.779999,133.899994,133.899994,329
2,2022-12-23 09:02:00,133.960007,134.039993,133.960007,134.039993,134.039993,746
3,2022-12-23 09:03:00,134.020004,134.020004,133.979996,133.979996,133.979996,100
4,2022-12-23 09:04:00,133.960007,133.960007,133.860001,133.860001,133.860001,569
...,...,...,...,...,...,...,...
255,2022-12-23 13:26:00,133.440002,133.440002,133.440002,133.440002,133.440002,133
256,2022-12-23 13:27:00,133.419998,133.419998,133.399994,133.399994,133.399994,227
257,2022-12-23 13:28:00,133.419998,133.419998,133.399994,133.419998,133.419998,158
258,2022-12-23 13:29:00,133.440002,133.460007,133.440002,133.460007,133.460007,81


In [18]:
sigma = {}
for ticker in cac40_tickers:
    query = session.query(cac40_classes[ticker])
    temp_df = pd.read_sql(query.statement, query.session.bind, index_col='Datetime')
    sigma[ticker] = df.Close.pct_change().std()

sigma

{'AI.PA': 0.0004388542756115271,
 'AIR.PA': 0.0004388542756115271,
 'ALO.PA': 0.0004388542756115271,
 'MT.AS': 0.0004388542756115271,
 'CS.PA': 0.0004388542756115271,
 'BNP.PA': 0.0004388542756115271,
 'EN.PA': 0.0004388542756115271,
 'CAP.PA': 0.0004388542756115271,
 'CA.PA': 0.0004388542756115271,
 'ACA.PA': 0.0004388542756115271,
 'BN.PA': 0.0004388542756115271,
 'DSY.PA': 0.0004388542756115271,
 'ENGI.PA': 0.0004388542756115271,
 'EL.PA': 0.0004388542756115271,
 'ERF.PA': 0.0004388542756115271,
 'RMS.PA': 0.0004388542756115271,
 'KER.PA': 0.0004388542756115271,
 'OR.PA': 0.0004388542756115271,
 'LR.PA': 0.0004388542756115271,
 'MC.PA': 0.0004388542756115271,
 'ML.PA': 0.0004388542756115271,
 'ORA.PA': 0.0004388542756115271,
 'RI.PA': 0.0004388542756115271,
 'PUB.PA': 0.0004388542756115271,
 'RNO.PA': 0.0004388542756115271,
 'SAF.PA': 0.0004388542756115271,
 'SGO.PA': 0.0004388542756115271,
 'SAN.PA': 0.0004388542756115271,
 'SU.PA': 0.0004388542756115271,
 'GLE.PA': 0.0004388542756

In [49]:
update = {ticker: session.query(cac40_classes[ticker]).order_by(
    cac40_classes[ticker].Datetime.desc()).first().Datetime for ticker in cac40_tickers}

while True:
    for ticker in cac40_tickers:
        new_data = yf.download(tickers=ticker,
                               period="1d",
                               interval="1m", progress=False).reset_index()
        new_data = new_data[new_data['Datetime'] > update[ticker]]
        for _, tick in new_data.iterrows():
            last_close = session.query(cac40_classes[ticker]).order_by(
                cac40_classes[ticker].Datetime.desc()).first().Close
            change_ratio = (tick["Close"] / last_close - 1) / sigma[ticker]
            if change_ratio > 2:
                print(f'{ticker} stock price moved by {(tick["Close"] / last_close - 1) * 100:.2f}% '
                      f'({change_ratio:.2f}x volatility) '
                      f'from {update[ticker].strftime("%H:%M")} '
                      f'to {tick["Datetime"].strftime("%H:%M")}.')
                #sleep(5)
            t = cac40_classes[ticker](**tick.to_dict())
            session.add(t)
            session.commit()
            update[ticker] = tick['Datetime']

AIR.PA stock price moved by 0.09% (2.04x volatility) from 13:43 to 13:44.
AIR.PA stock price moved by 0.09% (2.04x volatility) from 14:07 to 14:09.
AIR.PA stock price moved by 0.09% (2.04x volatility) from 14:29 to 14:30.
AIR.PA stock price moved by 0.14% (3.27x volatility) from 14:33 to 14:34.
AIR.PA stock price moved by 0.21% (4.88x volatility) from 14:38 to 14:39.
AIR.PA stock price moved by 0.11% (2.44x volatility) from 14:41 to 14:42.
AIR.PA stock price moved by 0.16% (3.68x volatility) from 15:30 to 15:31.
AIR.PA stock price moved by 0.09% (2.05x volatility) from 15:36 to 15:37.
AIR.PA stock price moved by 0.09% (2.05x volatility) from 15:57 to 15:58.
AIR.PA stock price moved by 0.09% (2.04x volatility) from 16:07 to 16:08.
AIR.PA stock price moved by 0.09% (2.04x volatility) from 16:09 to 16:10.
AIR.PA stock price moved by 0.09% (2.04x volatility) from 16:10 to 16:11.
AIR.PA stock price moved by 0.11% (2.45x volatility) from 16:15 to 16:16.
AIR.PA stock price moved by 0.09% (2.0

KeyboardInterrupt: 

In [45]:
print(update['AI.PA'].strftime("%H:%M"))

13:30


In [46]:
query = session.query(cac40_classes['AIR.PA']).order_by(
    cac40_classes['AIR.PA'].Datetime.desc()).limit(10)
pd.read_sql(query.statement, query.session.bind, index_col='Datetime')

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-23 13:30:00,111.599998,111.599998,111.599998,111.599998,111.599998,0
2022-12-23 13:29:00,111.559998,111.580002,111.559998,111.580002,111.580002,138
2022-12-23 13:28:00,111.500000,111.500000,111.500000,111.500000,111.500000,0
2022-12-23 13:27:00,111.540001,111.540001,111.540001,111.540001,111.540001,61
2022-12-23 13:24:00,111.540001,111.540001,111.540001,111.540001,111.540001,48
2022-12-23 13:23:00,111.500000,111.500000,111.500000,111.500000,111.500000,38
2022-12-23 13:22:00,111.480003,111.500000,111.459999,111.500000,111.500000,93
2022-12-23 13:21:00,111.500000,111.500000,111.480003,111.480003,111.480003,376
2022-12-23 13:20:00,111.480003,111.480003,111.480003,111.480003,111.480003,89


In [ ]:
dt = session.query(cac40_classes['AI.PA']).order_by(
    cac40_classes['AI.PA'].Datetime.desc()).first().Datetime
dt

In [51]:
session.query(cac40_classes['AI.PA']).order_by(cac40_classes['AI.PA'].Datetime.desc()).first().Close

133.82000732421875